## 보배드림 크롤링
- 문제1: 날짜 지정은 어떻게 해야할까?
> `등록일`을 크롤링해서 날짜를 구분하자!!
> "/"로 split했을 때 list len이 2이면 월일 구분, 1이면 오늘
- 문제2: 몇 년도인지는 어떻게 구분할까?
> 게시글 안의 날짜 크롤링!
- 문제3: 댓글의 필요없는 앞뒤 공백들을 어떻게 없앨까?
> `strip` 함수 사용!!
- 문제4: 댓글 크롤링시 댓글, 대댓글이 같이 긁어져옴
- 문제5: 댓글이 아주 많아서 댓글 페이지가 바뀌는 경우에는??(자바스크립트로 되어있음)

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# 게시판 url 지정
base_url = "http://www.bobaedream.co.kr"
page_url1 = "/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key="
page_url2 = "&page="

# 검색어 & page
search = "호환"
page_num = "1"

url = base_url + page_url1 + search + page_url2 + page_num
url

'http://www.bobaedream.co.kr/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key=호환&page=1'

In [3]:
# 해당 검색어에 대한 게시글의 html 객체 가져오기
req = requests.get(url).content
doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")
# doc

In [4]:
# 게시글 url 가져오기
article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
len(article_list)
# article_url = [ i.get("href") for i in article_list ]
# article_url
# len(article_url)

98

In [5]:
# 날짜 기져오기(첫 페이지에서 best글들은 제외!!)
date = [ i.select("td.date")[0].text  for i in doc.select("#boardlist > tbody > tr") if len(i.select("td.num01")) == 1 ]
date

['02/19',
 '02/16',
 '02/09',
 '02/03',
 '01/29',
 '01/28',
 '01/28',
 '01/23',
 '01/22',
 '01/19',
 '01/14',
 '01/11',
 '01/11',
 '01/08',
 '01/05',
 '12/31',
 '12/31',
 '12/30',
 '12/29',
 '12/17',
 '12/12',
 '12/10',
 '12/10',
 '12/09',
 '11/23',
 '11/22',
 '11/22',
 '11/22',
 '11/04',
 '10/01',
 '09/27',
 '09/13',
 '09/06',
 '08/22',
 '08/17',
 '07/02',
 '06/21',
 '06/16',
 '06/04',
 '05/24',
 '04/17',
 '04/06',
 '04/02',
 '02/24',
 '02/04',
 '09/10',
 '09/01',
 '08/30',
 '07/04',
 '03/31',
 '03/17',
 '03/14',
 '02/22',
 '02/14',
 '02/13',
 '01/29',
 '01/09',
 '12/26',
 '12/17',
 '12/14',
 '12/10',
 '11/26',
 '11/18',
 '11/13',
 '10/04',
 '09/09',
 '08/21',
 '07/12',
 '06/28',
 '06/18',
 '06/14',
 '05/18',
 '04/05',
 '01/25',
 '12/04',
 '12/02',
 '09/11',
 '07/15',
 '05/17',
 '04/27',
 '03/31',
 '03/30',
 '02/20',
 '01/13',
 '11/24',
 '10/27',
 '10/27',
 '10/07',
 '09/11',
 '06/28',
 '06/24',
 '05/20',
 '04/01',
 '02/18',
 '01/10',
 '01/09',
 '12/09',
 '10/24',
 '10/19',
 '09/21']

In [6]:
# 게시글 제목, 내용, 댓글 가져오기
url_article = base_url + "/view?code=freeb&No=1554408&bm=1"
req = requests.get(url_article).content
doc = BeautifulSoup(req, "html.parser")

In [25]:
subject = "".join( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")[:-1] )
content = doc.select("#print_area > div.content02 > div")[0].text
comment_list = doc.select("li > dl > dd")
comment = [ i.text.strip() for i in comment_list ]   # 댓글에 있는 앞뒤 공백 제거
yymmdd = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]
yyyymm = "".join( yymmdd[:8].split(".") )

In [8]:
subject, content, comment[0], yymmdd

('나쁜 사람들이 정말 많네요',
 '\n지겨운 월요일 잘 보내고 계신가요???저는 컴퓨터 잘 모릅니다.컴퓨터가 안켜져서 출장수리를 불렀는데 메인보드가 나가서 갈아야 한다네요출장 불렀더니 12만원 달랍니다.출장비 3만원을 쳐주더라도 9만원이 메인보드 값이라는 건데 중고에, 기존제품 돌려주지도 않고...ㅎ...그와중에 알고보니\xa0메인보드가 기존에 쓰던거보다 하위 제품이고, 호환이 안된다고 cpu까지 하위 제품으로\xa0바꿔놓고 1도 언급은 없고결국은 동생이 나서서 6만원 돌려받고, 메인보드, cpu 다 돌려받았습니다만좋게 얘기해서는 말이 안통하네요..거기서 바꿔준 중고\xa0메인보드+ 중고 cpu 해도 3만원이면 될텐데..우리 메인보드 + cpu 돌려줄 생각도 없었으면서 돈도\xa012만원 까지 받아 챙길려고 하다니양심적으로 삽시다..아무리 먹고 살기 위해서 하는 장사라지만 사람이 양심은 있어야죠...?\n',
 '차카게 삽시다 ~ ㅎ',
 '2019.01.14')

In [3]:
# for문으로 검색어에 대한 게시글 url 가져오기
article_url = []
date = []
for page in range(1,10,1):
    # 게시판 url 지정
    base_url = "http://www.bobaedream.co.kr"
    page_url1 = "/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key="
    page_url2 = "&page="

    # 검색어 & page
    search = "호환"
    page_num = str(page)

    url = base_url + page_url1 + search + page_url2 + page_num
    
    # 해당 검색어에 대한 게시글의 html 객체 가져오기
    req = requests.get(url).content
    doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")
    
    # 게시글 url 가져오기
    article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
    if len(article_list) == 0:
        break
    article_url = article_url + [ i.get("href") for i in article_list ]

    # 날짜 기져오기(best글, 관리자에 의해 삭제된 글은 제외!!)
    date = date + [ i.select("td.date")[0].text  for i in doc.select("#boardlist > tbody > tr") if len(i.select("td.pl14 > a.bsubject")) == 1 ]
    
    print(str(page) + "페이지 완료")

1페이지 완료
2페이지 완료
3페이지 완료


In [51]:
len(article_url), len(date)

(212, 212)

In [59]:
# for문으로 전체 url에 대한 글 크롤링
subject = []
content = []
comment = []
yymmdd = []
for sub_url in article_url:
    # 게시글 제목, 내용, 댓글 가져오기
    url_article = base_url + sub_url
    req = requests.get(url_article).content
    doc = BeautifulSoup(req, "html.parser")

    date = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]
    if int(date[:4]) <= 2016:
        print("{}개 데이터 크롤링 완료!".format( len(yymmdd) ))
        break
    else:
        yymmdd.append(date)
        subject.append( "".join( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")[:-1] ) )
        content.append( doc.select("#print_area > div.content02 > div")[0].text )
        comment_list = doc.select("li > dl > dd")
        comment.append( [ i.text.strip() for i in comment_list ] )   # 댓글에 있는 앞뒤 공백 제거

56개 데이터 크롤링 완료!


In [60]:
len(subject), len(content), len(comment), len(yymmdd)

(56, 56, 56, 56)

In [32]:
# 보배드림에서 검색어에 대한 게시글 url을 가져오는 함수 => date는 가져올 필요 없을듯??
def get_url_all(search):
    # for문으로 검색어에 대한 게시글 url 가져오기
    article_url = []
    date = []
    for page in range(1, 1000, 1):
        # 게시판 url 지정
        base_url = "http://www.bobaedream.co.kr"
        page_url1 = "/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key="
        page_url2 = "&page="

        # 검색어 & page
        search = search
        page_num = str(page)

        url = base_url + page_url1 + search + page_url2 + page_num

        # 해당 검색어에 대한 게시글의 html 객체 가져오기
        req = requests.get(url).content
        doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")

        # 게시글 url 가져오기
        article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
        if len(article_list) == 0:
            print("작업 완료")
            break
        article_url = article_url + [ i.get("href") for i in article_list ]

        # 날짜 기져오기(best글, 관리자에 의해 삭제된 글은 제외!!)
        date = date + [ i.select("td.date")[0].text  for i in doc.select("#boardlist > tbody > tr") if len(i.select("td.pl14 > a.bsubject")) == 1 ]

        print(str(page) + "페이지 완료")
        
    return article_url, date


# 각 게시글의 제목, 내용, 댓글을 크롤링하는 함수
def get_crawling_data(article_url):
    # for문으로 전체 url에 대한 글 크롤링
    subject = []
    content = []
    comment = []
    yymmdd = []
    yyyymm = []
    for ind, sub_url in enumerate(article_url):
        # 게시글 제목, 내용, 댓글 가져오기
        url_article = base_url + sub_url
        req = requests.get(url_article).content
        doc = BeautifulSoup(req, "html.parser")
        
        date = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]
        if int(date[:4]) <= 2016:
            break
        else:
            yymmdd.append(date)
            yyyymm.append( "".join( date[:8].split(".") ) )
            subject_text = doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")
            if len(subject_text) == 1:
                subject.append(subject_text[0])
            else:
                subject.append( "".join(subject_text[:-1] ) )
#             subject.append( "".join( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")[:-1] ) )
#             subject.append( doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text )
            content.append( doc.select("#print_area > div.content02 > div")[0].text )
            comment_list = doc.select("li > dl > dd")
            comment.append( ",".join( [ i.text.strip() for i in comment_list ]) )   # 댓글에 있는 앞뒤 공백 제거    
        
        # 게시글 10개마다 개수 확인
        if (ind + 1) % 10 == 0:
            print("{}개 url 완료".format(ind + 1))
            
    print("{}개 데이터 크롤링 완료!".format(len(yymmdd)))
    
    import pandas as pd

    result = pd.DataFrame( {"yyyymm": yyyymm,
                            "date": yymmdd,
                            "subject": subject,
                            "content": content,
                            "comment": comment} )
    
    pd.to_csv("{}_crawling_data.csv".format())
    
    return subject, content, comment, yymmdd, yyyymm, result

In [29]:
# 검색어에 대한 게시글 url 가져오기
article_url, date = get_url_all("호환")
len(article_url), len(date)

1페이지 완료
2페이지 완료
3페이지 완료
작업 완료


(211, 211)

In [34]:
# 게시글에 대한 제목, 내용, 댓글 크롤링
subject, content, comment, yymmddm, yyyymm, crawler_data = get_crawling_data(article_url)
len(subject), len(content), len(comment), len(yyyymm)

10개 url 완료
20개 url 완료
30개 url 완료
40개 url 완료
50개 url 완료
55개 데이터 크롤링 완료!


(55, 55, 55, 55)

In [4]:
# # 보배드림에서 검색어에 대한 게시글 url을 가져오는 함수 => date는 가져올 필요 없을듯??
# # 검색어에 대응하는 각 게시글의 제목, 내용, 댓글을 크롤링하는 함수
# def get_crawling_data(search):
#     # for문으로 검색어에 대한 게시글 url 가져오기
#     article_url = []
#     date = []
#     for page in range(1, 1000, 1):
#         # 게시판 url 지정
#         base_url = "http://www.bobaedream.co.kr"
#         page_url1 = "/list.php?code=freeb&or_gu=10&or_se=desc&pagescale=100&s_key="
#         page_url2 = "&page="

#         # 검색어 & page
#         search = search
#         page_num = str(page)

#         url = base_url + page_url1 + search + page_url2 + page_num

#         # 해당 검색어에 대한 게시글의 html 객체 가져오기
#         req = requests.get(url).content
#         doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")

#         # 게시글 url 가져오기
#         article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
#         if len(article_list) == 0:
#             print("작업 완료")
#             break
#         article_url = article_url + [ i.get("href") for i in article_list ]

# #         # 날짜 기져오기(best글, 관리자에 의해 삭제된 글은 제외!!)
# #         date = date + [ i.select("td.date")[0].text  for i in doc.select("#boardlist > tbody > tr") if len(i.select("td.pl14 > a.bsubject")) == 1 ]

#         print(str(page) + "페이지 완료")
        
#     print("crawling 시작!")
#     # for문으로 전체 url에 대한 글 크롤링
#     subject = []
#     content = []
#     comment = []
#     yymmdd = []
#     yyyymm = []
#     for ind, sub_url in enumerate(article_url):
#         # 게시글 제목, 내용, 댓글 가져오기
#         url_article = base_url + sub_url
#         req = requests.get(url_article).content
#         doc = BeautifulSoup(req, "html.parser")
        
#         date = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]
#         if int( "".join( date[:10].split(".") ) ) <= 20160311:
#             break
#         else:
#             yymmdd.append( "".join( date[:10].split(".") ) )
#             yyyymm.append( "".join( date[:8].split(".") ) )
#             subject_text = doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")
#             if len(subject_text) == 1:
#                 subject.append(subject_text[0])
#             else:
#                 subject.append( "".join(subject_text[:-1] ) )
#             content.append( doc.select("#print_area > div.content02 > div")[0].text )
#             comment_list = doc.select("li > dl > dd")
#             comment.append( ",".join( [ i.text.strip() for i in comment_list ]) )   # 댓글에 있는 앞뒤 공백 제거    
        
#         # 게시글 10개마다 개수 확인
#         if (ind + 1) % 10 == 0:
#             print("{}개 url 완료".format(ind + 1))
            
#     print("{}개 데이터 크롤링 완료!".format(len(yymmdd)))
    
#     import pandas as pd

#     result = pd.DataFrame( {"yyyymm": yyyymm,
#                             "date": yymmdd,
#                             "subject": subject,
#                             "content": content,
#                             "comment": comment} )
#     result.to_csv("{}_crawling_data.csv".format(search), index=False)
    
#     return result

In [13]:
from datetime import datetime  
int(datetime.today().strftime("%Y%m"))

201903

# 완성된 크롤링 코드

In [158]:
# 보배드림에서 자유게시판, 국내차게시판, 수입차게시판에서 검색어에 대한 게시글 url을 가져오는 함수
# 검색어에 대응하는 각 게시글의 제목, 내용, 댓글을 크롤링하는 함수
def get_crawling_data(search):
    from bs4 import BeautifulSoup
    import requests
    from datetime import datetime    
    import pandas as pd
    
    subject = []
    content = []
    comment = []
    yymmdd = []
    yyyymm = []
    
    for board_name in ["freeb", "national", "import"]:
        # for문으로 검색어에 대한 게시글 url 가져오기
        article_url = []
        date = []
        for page in range(1, 1000, 1):
            # 게시판 url 지정
            base_url = "http://www.bobaedream.co.kr"
            page_url1 = "/list.php?code="
            page_url2 = "&or_gu=10&or_se=desc&pagescale=100&s_key="
            page_url3 = "&page="

            # 검색어 & page
            search = search
            page_num = str(page)

            url = base_url + page_url1 + board_name + page_url2 + search + page_url3 + page_num

            # 해당 검색어에 대한 게시글의 html 객체 가져오기
            req = requests.get(url).content
            doc = BeautifulSoup(req, "html.parser", from_encoding="utf-8")

            # 게시글 url 가져오기
            article_list = doc.select("#boardlist > tbody > tr > td.pl14 > a.bsubject")
            if len(article_list) == 0:
                print("작업 완료")
                break
            article_url = article_url + [ i.get("href") for i in article_list ]
            
            print(str(page) + "페이지 완료")

        print("crawling 시작!")
        
        # for문으로 전체 url에 대한 글 크롤링
        expression = ["\n", "\t", "\r", "\xa0"]
        
        for ind, sub_url in enumerate(article_url):
            # 게시글 제목, 내용, 댓글 가져오기
            url_article = base_url + sub_url
            req = requests.get(url_article).content
            doc = BeautifulSoup(req, "html.parser")

            date = doc.select("#print_area > div.writerProfile > dl > dt > span")[0].text.split("|")[2].replace("\xa0", " ").strip()[:10]
            if int( "".join( date[:7].split(".") ) ) < int(datetime.today().strftime("%Y%m")) - 300:   # 오늘부터 3년전의 날짜
                break
            else:
                subject_text = doc.select("#print_area > div.writerProfile > dl > dt > strong")[0].text.split("[")
                if len(subject_text) == 1:
                    subject_text = subject_text[0]
                else:
                    subject_text = "".join(subject_text[:-1])
                content_text = doc.select("#print_area > div.content02 > div")[0].text
                comment_list = doc.select("li > dl > dd")
                comment_text = ",".join( [ i.text.strip() for i in comment_list ])   # 댓글에 있는 앞뒤 공백 제거
                for ex in expression:
                    subject_text = subject_text.replace(ex, " ")  
                    content_text = content_text.replace(ex, " ")  
                    comment_text = comment_text.replace(ex, " ")  
                # 댓글이 하나도 없으면 데이터에 추가 X
                if len(comment_text) != 0:
                    yymmdd.append( "".join( date[:10].split(".") ) )
                    yyyymm.append( "".join( date[:8].split(".") ) )
                    subject.append(subject_text.strip())
                    content.append(content_text.strip())
                    comment.append(comment_text.strip())     
                
            # 게시글 10개마다 개수 확인
            if (ind + 1) % 10 == 0:
                print("{}개 url 완료".format(ind + 1))

        print("[{}]게시판 {}개 데이터 크롤링 완료!".format(board_name, len(yymmdd)))
    
    result = pd.DataFrame( {"yyyymm": yyyymm,
                            "date": yymmdd,
                            "subject": subject,
                            "content": content,
                            "comment": comment} )
    result.to_csv("{}_crawling_data.csv".format(search), index=False, encoding="utf-8")

    # 월별로 모아놓은 데이터
    # 201603 ~ 201902까지의 데이터
    month_list = [ str(i)+"0"+str(j) if len(str(j))!=2 else str(i)+str(j) for i in range(2016,2020,1) for j in range(1,13,1) ][2:(36+2)]
    yyyymm = []
    subject_month = []
    content_month = []
    comment_month = []
    count_list = []
    
    for month in month_list:
        subject = ""
        content = ""
        comment = ""
        count = 0
        for ind in range(result.shape[0]):
            if result["yyyymm"][ind] == month:
                subject = subject + " " + result["subject"][ind]
                content = content + " " + result["content"][ind]
                comment = comment + " " + result["comment"][ind]
                count = count + 1
#             if (ind == result.shape[0]-1) & (len(subject) > 0):    # 연월이 바뀌고 글이 존재할경우 append하겠다!
            if ind == result.shape[0]-1:
                yyyymm.append(month)
                subject_month.append(subject.strip())
                content_month.append(content.strip())
                comment_month.append(comment.strip())
                count_list.append(count)
                break

    result2 = pd.DataFrame({"yyyymm": yyyymm,
                            "subject": subject_month,
                            "content": content_month,
                            "comment": comment_month,
                            "count": count_list})
    result2.to_csv("{}_crawling_data_monthly.csv".format(search), index=False, encoding="utf-8")
    
    return result, result2

In [159]:
crawler_data, data2 = get_crawling_data("호환")

1페이지 완료
2페이지 완료
3페이지 완료
작업 완료
crawling 시작!
10개 url 완료
20개 url 완료
30개 url 완료
40개 url 완료
50개 url 완료
60개 url 완료
70개 url 완료
[freeb]게시판 58개 데이터 크롤링 완료!
1페이지 완료
2페이지 완료
3페이지 완료
작업 완료
crawling 시작!
10개 url 완료
20개 url 완료
30개 url 완료
40개 url 완료
50개 url 완료
60개 url 완료
70개 url 완료
80개 url 완료
90개 url 완료
100개 url 완료
110개 url 완료
120개 url 완료
130개 url 완료
[national]게시판 182개 데이터 크롤링 완료!
1페이지 완료
작업 완료
crawling 시작!
10개 url 완료
[import]게시판 194개 데이터 크롤링 완료!


In [160]:
crawler_data.shape

(194, 5)

In [161]:
crawler_data.head()

,yyyymm,date,subject,content,comment
0,201903,20190309,콤퓨타좀 여쭤볼께유,요 메인보드가 i5 2500k였던가 고거랑 디디알3 램 8기가 4개랑 그래픽 9...,컴에 대해서 어느정도 아시묜 도전하셔유.. 잘 모르시면 굳이 부품을 구하실 이유는...
1,201902,20190219,차량 트렁크,제 차량이 2007년 쎄라토 차량인데 트렁크가 찌그러져서 갈아 끼울려고 하는데 20...,"될거같은데 도전!,안될것같은데유 ㄷㄷ 된다고해도 보기싫을꺼에유 ㄷㄷㄷ,만약 된다고 ..."
2,201902,20190216,무책임한 아이코스 회사,안녕하세요 너무 화가나는 일이 있어 이렇게 글을 올려요 제가 2달전 18년 12...,"도난 한건 본인 책임이고, 재고재품 분명있고 아이코스가 리퍼개념이라 분명 리퍼제품도..."
3,201902,20190209,질문} 샤시 롤러 규격 질문 드립니다.,뻘질문 죄송합니다.샤시가 오래되어 뜯어보니 롤러 수명이 다된거 같습니다.(내부 베이...,"엘지 하이샤시 대리점으로 문의하세요. 시중에 물건 없습니다.,답글. 감사합니다.,..."
4,201902,20190203,형님 동생분들 요즘 컴퓨터 어떻게들 사시는지요?!,컴퓨터 새로 살려고 하는데 조립컴퓨터 조합하려고해도 각 부품호환등 이런게 너무 복...,"그냥 인터넷 다나와,아 들어본적있어요 검색해봐야겠네요,돈주고 삽니다 ㅋㅋㅋ 아는 동..."


In [162]:
crawler_data.tail()

,yyyymm,date,subject,content,comment
189,201806,20180605,e클 s클 휠 호환,제가 e클 11년식타고 245/45 17인치인데 주위에 s클 245/50 18인치...,"아뇨 차이커요 타이어는 바꾸면 가능 은함,타이어를 245 40 18 로 바꾸시면 s..."
190,201708,20170802,프리우스 3세대 에어컨필터 호환되는거 추천부탁드립니다,올뉼k5가 된다고하긴하는데 다른것은 없는지요?,"정비병 출신 없냐,Sm3 sm5뉴임프레셔 sm7 이마트에서 도요타렉서스 필터 팝니..."
191,201701,20170111,435d 에 m4순정 휠 호환되나요?ㅜㅜ,435d 에 m4순정 휠 호환되나요?원래 껴져있던 런플렛 타이어는 안껴지죠?ㅜㅜ휠타...,"호환됩니다~~,넙죽 감사합니다^^,근데 m2 랑 435d 랑 뭐가 더 잘나갈까요 '..."
192,201606,20160618,인피니티 호환문의,인피니티 G35랑 G37오디오cd룸 호환되는지 알고싶습니다,"오디오 튜닝샵에 전화헤서 물어보는게 가장 빠를듯 하네요 ㅎㅎ,아무 오디오 튜닝샵에..."
193,201605,20160504,휠 호환 여부 판단 부탁 드립니다.,차량은 E400 카브리올레 입니다.. 윈터셋트 하나 만들어놓으려고 휠 구하고있는데요...,E 클 휠은 씨클에 안맞고 e클 쿠페나 카브리 휠이 c 클에 맞습니다. 옵셋을 다 ...


In [163]:
data2.shape

(36, 5)

In [164]:
data2.head()

,yyyymm,subject,content,comment,count
0,201603,LF쏘나타 2.0 CVVL 8인치 순정네비+루카스 블박 OBD2 호환성질문 싼타페d...,2016년식 LF쏘나타 2.0 CVVL 8인치 순정네비 + 루카스 블박 LK-919...,QXD950의 OBD2 를 쓰면 블루링크의 일부기능을 못 쓴다????? 헐 요기서 ...,3
1,201604,아이폰 호환관련 ㅠㅠ 그랜저hg랑 아슬란 쇼바가 호환될까요? 라세티 부품호환가능한가...,안녕하세요^^며칠 전 06년식 아반떼hd 데려왔는데요 !ipod usb가 있어서 ...,급속충전usb케이블로 꽂으면 될지도 모릅니다 (아이패드 충전되는 usb케이블) ...,4
2,201605,벤츠 이클이랑 체어맨이랑 용품호환 대나바요? 투싼 isg 오토홀드 호환되나요??? ...,7단미션 미션필터 호환되나부네요 ㄷㄷ 돈굳겟.. 이번에 투싼계약을 했습니다 ...,"변속기가 같은거니까요.,완전같은가보네요..,오일같은것도 일부러 쌍용꺼 쓰는 사람이 ...",7
3,201606,tg그랜져 브레이크 뉴오피러스 브레이크호환문의드립니다 인피니티 호환 현대 순정 시가...,안녕하세요 브레이크가 말썽이네요 ㅜ 아는지인이 탈거히니놓은 tg 06년lpi모델순...,"로터,싸이즈 같다면??? 어라??? 패드가 틀린데?? 낼밤에,,한번더 올리면 답...",5
4,201607,블박 회사가 달라도 케이블은 호환되겠죠?,이전 블박 다본다히어로 수명이 다되어 아이나비 QXD950 View를 주문하고 기...,"재설치하믄 좋쥬,,, 보통 구뇽이 안맞는경우가 많어유 흐흐,같은 회사 제품이면 괜찮...",1


In [165]:
data2.tail()

,yyyymm,subject,content,comment,count
31,201810,단말기 호환 문의 아반떼xd 순정오디오 호환가능하지좀 알려주세요 리어 와이퍼의 호환성,단말기 호환 문의좀 드립니다.현재 skt 사용중입니다.단말기 중고로 갤럭시s9+로...,"상과없을거 같습니다,감사합니다 걍 써도 된다는 말씀이군요,전혀 상관없습니다 요즘폰은...",3
32,201811,블랙핑크는 투애니원 역대 노래 100% 호환 가능하지 않을까요 벨로스터구형수동 미션...,근데 들리는 말로는 아류 소리 듣기 싫어서 일부러 안한다는 얘기도 있구요 안녕하세요...,관심이 그다지 없는 내용이라...ㅠ.ㅠ 넵! 한번 찾아보도록 하겠습니다. 직접 까봐...,9
33,201812,Usb로tv연결해서보는거 도움부탁드립니다 아 배고파 YF소나타 사제 내비 매립 혹시...,큰맘먹고 10년 넘은티비 바꿨는데요 75인치 신세계네요~~ㅋㅋ 영화보믄 좋겠다싶...,"코덱,정보가 부족하여 pass.. TV 모델이라도 올려줘보세요..,아~~지금집이아니...",14
34,201901,Nf쏘나타 lpg차량 obd 호환 인터넷거래로 정품시켰는데 비슷하게생긴 싸구려가왔네...,갑자기 차량 냉각수온 계기판이 H 끝까지 올라갑니다. 써모스텟이랑 수온센서 모두 ...,"곤치는 방법 알려드릴까요?,싸구려 obd도 냉각수 온도는 나옵니다,계기판 얼마안하는...",29
35,201902,차량 트렁크 무책임한 아이코스 회사 질문} 샤시 롤러 규격 질문 드립니다. 형님 동...,제 차량이 2007년 쎄라토 차량인데 트렁크가 찌그러져서 갈아 끼울려고 하는데 20...,"될거같은데 도전!,안될것같은데유 ㄷㄷ 된다고해도 보기싫을꺼에유 ㄷㄷㄷ,만약 된다고 ...",18


In [166]:
data2[data2["count"] == max(data2["count"])]

,yyyymm,subject,content,comment,count
34,201901,Nf쏘나타 lpg차량 obd 호환 인터넷거래로 정품시켰는데 비슷하게생긴 싸구려가왔네...,갑자기 차량 냉각수온 계기판이 H 끝까지 올라갑니다. 써모스텟이랑 수온센서 모두 ...,"곤치는 방법 알려드릴까요?,싸구려 obd도 냉각수 온도는 나옵니다,계기판 얼마안하는...",29


In [12]:
crawler_data.isna().sum()   # 결측치 없음!!(but comment의 경우는 값이 없는 경우가 있으나 빈칸으로 들어가있음)

yyyymm     0
date       0
subject    0
content    0
comment    0
dtype: int64

In [82]:
import pandas as pd
data = pd.read_csv("호환_crawling_data.csv", engine="python", encoding="utf-8")
data.head()

,yyyymm,date,subject,content,comment
0,201903,20190311,폭스바겐 파사트 중고부품 호환성 여부 질문이요! 도와주세요ㅠ,안녕하세요 제가 폭스바겐 파사트 헤드라이트가 고장나서 DIY를 하고 있습니다.제 차...,NaN
1,201903,20190309,콤퓨타좀 여쭤볼께유,요 메인보드가 i5 2500k였던가 고거랑 디디알3 램 8기가 4개랑 그래픽 9...,컴에 대해서 어느정도 아시묜 도전하셔유.. 잘 모르시면 굳이 부품을 구하실 이유는...
2,201903,20190308,폭스바겐 파사트 DIY중인데 질문이 있습니다 도와주세요! ㅠ,안녕하세요! 폭스바겐 2007년식 파사트를 타고 있는 사람입니다.라이트가 고장이나서...,NaN
3,201903,20190307,필립스 전기 면도기 아이코닉 쓰시는분 있나요?,아이코닉 전기 면도기 쓰는데 날이 다 된건지 트러블이 자꾸 생기네요 모델명이 s...,NaN
4,201902,20190219,차량 트렁크,제 차량이 2007년 쎄라토 차량인데 트렁크가 찌그러져서 갈아 끼울려고 하는데 20...,"될거같은데 도전!,안될것같은데유 ㄷㄷ 된다고해도 보기싫을꺼에유 ㄷㄷㄷ,만약 된다고 ..."


In [56]:
data.where(data["yyyymm"]==201903)

,yyyymm,date,subject,content,comment
0,201903.0,20190311.0,폭스바겐 파사트 중고부품 호환성 여부 질문이요! 도와주세요ㅠ,안녕하세요 제가 폭스바겐 파사트 헤드라이트가 고장나서 DIY를 하고 있습니다.제 차...,NaN
1,201903.0,20190309.0,콤퓨타좀 여쭤볼께유,요 메인보드가 i5 2500k였던가 고거랑 디디알3 램 8기가 4개랑 그래픽 9...,컴에 대해서 어느정도 아시묜 도전하셔유.. 잘 모르시면 굳이 부품을 구하실 이유는...
2,201903.0,20190308.0,폭스바겐 파사트 DIY중인데 질문이 있습니다 도와주세요! ㅠ,안녕하세요! 폭스바겐 2007년식 파사트를 타고 있는 사람입니다.라이트가 고장이나서...,NaN
3,201903.0,20190307.0,필립스 전기 면도기 아이코닉 쓰시는분 있나요?,아이코닉 전기 면도기 쓰는데 날이 다 된건지 트러블이 자꾸 생기네요 모델명이 s...,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [95]:
subject = ""
content = ""
comment = ""
for ind in range(crawler_data.shape[0]):
    if crawler_data["yyyymm"][ind] == "201903":
        subject = subject + " " + crawler_data["subject"][ind]
        content = content + " " + crawler_data["content"][ind]
        comment = comment + " " + crawler_data["comment"][ind]

'  컴에 대해서 어느정도 아시묜 도전하셔유..  잘 모르시면 굳이 부품을 구하실 이유는 엄네유...  특히 보드는 사진짝어가묜서 작업해야대서..  손이좀 많이 가네유~~,컴맹인디 사망한컴이 메인보드가 문제라고 해소.......ㅠㅠ,@심심무료한놈 그렇게 진단하신분한테 부탁하셔유~~  교육비를 내면서 배워유~~~!!  처음부터 넘 어려운 코스네유...  부품중 메인보드는 낭중에 고수가 되어두 할까 말까네유~~,씨피유는 적으신게 맞다면 호환기종이 맞습니다. 램도 맞구요~ 파워만 부족허지않다면 그래픽도 잘 돌아갈듯합니다!,감싸합니다 형님 미사용재고라캐서 구입했습니다 낄낄,혹시라도 참고하셔유~~  자동차는 보배드림...  콤퓨타는 이토랜드네유~ http://www.etoland.co.kr/index.php  컴쪽 궁금한것은 이토님들이 좀 많이 잘해줘유~~  보배에두 이토하시는분들 상당수 될꺼네유~   네 저도 파란손 가서 했습니다,전 그냥 현대모비스 가서 사서 파란손에서 등록했습니다.,쇼트하시면서  리모콘키 누르셨나요,넵 철컥소리가 안나서 당황스럽네요~,호환이 되면 다행이네요.. 형님들 답변 감사합니다~~ 드릴로 한참깎아야됨,홀쏘랑 쐬톱 사와야긋네요 ㅠㅜ,도대체 왜 사서 고생하는지 이해 안됩니다 고장난것도 아니고 멀쩡한걸 구지;,지금꺼 넘모별로라서 욕심 좀 내봤읍니다...,까페 가입해서 정보 보세요,그게 빠를것같읍니다 ㅜㅜ,동일 제조사 상위등급 차량은 보통 가공없이 맞는데;;;;,이상하게도 안맞네요ㅜㅜ 세대가 다른차라 그런가...,sl꺼 그냥 들어가면 k9 호환 됩니다. sl에 k9꺼 장착했습니다.,SL꺼랑은 또 다른 모양입니다 ㄷㄷ 디블럭 한번써보세요,이런거 오프라인매장에서도 살수잇을가여 갑자기 당장갈고싶어지네요 ㄷㄷ,불스원 5중,PM 1.0 97%, PM 2.5 98%로 광고중인 두원 에코프리미엄활성탄 제품이나 PM 2.5 99%로 광고중인 3M F62시리즈.,보쉬 PM2.5필터욤. 카더라 통신은 믿을게 못됩니다.,같은 차대에 같은 파워트레인 쓰면서 디자인만 틀린

In [81]:
import numpy as np
data = data.replace(np.nan, "", regex=True)

In [27]:
x = data.iloc[0,3]
x

'\n안녕하세요 제가 폭스바겐 파사트 헤드라이트가 고장나서 DIY를 하고 있습니다.제 차는 2007년식 파사트 모델입니다.헤드라이트 통이 문제여서 통을 한 업체에서 주문을 했습니다. 운전석 쪽 헤드라이트 통이구요..그런데 제가 이 통을 장착하고 모듈이랑 안전기 다 장착하고 상향등을 켜보니 오른쪽만 나오고 새로 주문해서 단 라이트에선 헤드라이트가 나오질 않더군요.. 그래서 새로운 중고 통을 사서 장착을 해봤더니 잘 작동 되더라구요..이게 반품기간 7일이 지났긴 했지만 제 차와 호환성이 안맞는 것 같다는 이유로 환불신청을 했는데, 업체 쪽에선 통 자체는 문제가 없다고 환불은 안된다고 말하시더라구요. 업체측에서 판매한 제품이 호환성 여부는 검증된거라면서 2007년식 파사트에 모두 호환이 된다고 하더라구요.. 업체측에선 호환성여부 또한 체크해서 판매한거라고 하지만 제 차대번호도 업체측에서 모르고 핀매할 당시 재고가 하나 밖에 없으니, 입금을 빨리 하라고 한 적 또한 있습니다. 실제로 업체측이 판매한 통은 정상작동 되는 통은 맞는 것 같습니다. 하지만 제 차에 장착해보았을 때 정상작동이 안되고 호환성 여부를 고려하지 않고 보내준 업체 측에서도 책임이 있지 않나요??\xa0아니면 업체의 말대로 2007 파사트 헤드라이트 부품은 호완성 관계에 상관 없이 모든 2007 파사트에 호환이 되는 걸까요?후자의 경우 제 잘못을 인정하고 책임을 받아드리겠지만 만약 업체가 판매한 제품이 제 차와 호환성이 안되는 것이라면 환불요청이 정당하지 않나요?\xa0위쪽이 기존에 달려있던 파사트 통이고 아래쪽이 새로 구입한 통입니다.\n'

In [44]:
expression = ["\n", "\t", "\r", "\xa0"]
for ex in expression:
    x = x.replace(ex, " ")
# x.replace("\n\xa0", " ")
x

' 안녕하세요 제가 폭스바겐 파사트 헤드라이트가 고장나서 DIY를 하고 있습니다.제 차는 2007년식 파사트 모델입니다.헤드라이트 통이 문제여서 통을 한 업체에서 주문을 했습니다. 운전석 쪽 헤드라이트 통이구요..그런데 제가 이 통을 장착하고 모듈이랑 안전기 다 장착하고 상향등을 켜보니 오른쪽만 나오고 새로 주문해서 단 라이트에선 헤드라이트가 나오질 않더군요.. 그래서 새로운 중고 통을 사서 장착을 해봤더니 잘 작동 되더라구요..이게 반품기간 7일이 지났긴 했지만 제 차와 호환성이 안맞는 것 같다는 이유로 환불신청을 했는데, 업체 쪽에선 통 자체는 문제가 없다고 환불은 안된다고 말하시더라구요. 업체측에서 판매한 제품이 호환성 여부는 검증된거라면서 2007년식 파사트에 모두 호환이 된다고 하더라구요.. 업체측에선 호환성여부 또한 체크해서 판매한거라고 하지만 제 차대번호도 업체측에서 모르고 핀매할 당시 재고가 하나 밖에 없으니, 입금을 빨리 하라고 한 적 또한 있습니다. 실제로 업체측이 판매한 통은 정상작동 되는 통은 맞는 것 같습니다. 하지만 제 차에 장착해보았을 때 정상작동이 안되고 호환성 여부를 고려하지 않고 보내준 업체 측에서도 책임이 있지 않나요?? 아니면 업체의 말대로 2007 파사트 헤드라이트 부품은 호완성 관계에 상관 없이 모든 2007 파사트에 호환이 되는 걸까요?후자의 경우 제 잘못을 인정하고 책임을 받아드리겠지만 만약 업체가 판매한 제품이 제 차와 호환성이 안되는 것이라면 환불요청이 정당하지 않나요? 위쪽이 기존에 달려있던 파사트 통이고 아래쪽이 새로 구입한 통입니다. '

In [5]:
driver.get(url)

In [4]:
from selenium import webdriver

# 웹드라이브로 크롬브라우즈 띄운다.
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
driver.implicitly_wait(3)         # 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다

In [1]:
base_url = "http://kin.naver.com/search/list.nhn?&query="
per_url_1 = "&period="
per_url_2 = "%7C"
page_url = "&page="

query = "셧다운"
per1 = "2018.01.01."
per2 = "2018.01.23."
page = str(1)
driver.get(base_url + query + per_url_1 + per1 + per_url_2 + per2 + page_url + page)      # 네이버 지식인 1page로 이동

http://kin.naver.com/search/list.nhn?&query=셧다운&period=2018.01.01.%7C2018.01.23.&page=1


In [76]:
# 1 페이지의 url 가져오기
article_list = driver.find_elements_by_css_selector('dl > dt > a')
article_urls = [ i.get_attribute('href') for i in article_list ]
article_urls

['http://kin.naver.com/qna/detail.nhn?d1id=9&dirId=9020201&docId=292966571&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=1&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=4&dirId=409&docId=292962524&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=2&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=6&dirId=61302&docId=292965291&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=3&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=6&dirId=614&docId=292972107&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=4&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=4&dirId=40102&docId=293017325&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=5&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=8&dirId=814&docId=292719046&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=6&search_sort=0&spq=1',
 'http://kin.naver.com/qna/detail.nhn?d1id=9&dirId=9020201&docId=293007303&qb=7IWn64uk7Jq0&enc=utf8&section=kin&rank=7&search_sort=0&spq=1',
 'http://kin.naver.com/qna/de